### File to test OCR and LLM connection

In [1]:
# Import LLM support libraries
from langchain.llms import AzureOpenAI

# Import OCR support libraries
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from PIL import Image

# Import general libraries
from dotenv import load_dotenv, find_dotenv
import time
import os

In [2]:
# Load environment variables from the .env file
load_dotenv(find_dotenv())

# Retrieve Azure OpenAI specific configuration from environment variables
OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_API_TYPE = os.getenv("AZURE_OPENAI_API_TYPE")
OPENAI_API_BASE = os.getenv("AZURE_OPENAI_API_BASE")
OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
LLM_RESOURCE_NAME = os.getenv("LLM_RESOURCE_NAME")
COMPUTER_VISION_KEY = os.getenv("CV_KEY")
COMPUTER_VISION_ENDPOINT = os.getenv("CV_ENDPOINT")

In [3]:
# Initialise LLM object

# Initialize an instance of AzureOpenAI using the specified settings
model = AzureOpenAI(
    openai_api_version=OPENAI_API_VERSION,
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE,
    openai_api_type=OPENAI_API_TYPE,
    deployment_name="dptm-llm-gpt-35-turbo"
)

# Initialise computer vision object for OCR
computervision_client = ComputerVisionClient(COMPUTER_VISION_ENDPOINT, CognitiveServicesCredentials(COMPUTER_VISION_KEY))

/var/folders/qh/sckhy9cx1y13y4gg32cmhqv80000gn/T/ipykernel_4266/2301410953.py:4: LangChainDeprecationWarning: The class `AzureOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAI``.
  model = AzureOpenAI(
/Users/raveeshyadav/College/DPTM/dptmenv/lib/python3.9/site-packages/langchain_community/llms/openai.py:882: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://dptm-llm.openai.azure.com/ to https://dptm-llm.openai.azure.com/openai.
  warnings.warn(
/Users/raveeshyadav/College/DPTM/dptmenv/lib/python3.9/site-packages/langchain_community/llms/openai.py:889: UserWarning: As of openai>=1.0.0, if `deployment_name`

In [4]:
# Extract text from image using OCR
scannedLines = list()

# Open local image file
with open("sample_image.png", "rb") as image:
    read_response = computervision_client.read_in_stream(image, raw=True)
# Get the operation location (URL with an ID at the end)
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]
# Retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower() not in ['notstarted', 'running']:
        break
    time.sleep(1)
# Get the detected text
if read_result.status == OperationStatusCodes.succeeded:
    for page in read_result.analyze_result.read_results:
        for line in page.lines:
            # Print line
            scannedLines.append(line.text)
scannedText = "\n".join(scannedLines).replace('\n', ' ')

In [5]:
# Setting up prompt to summarise the text
prompt = "Summarise the following research in 30 words: "

# Adding the scanned text to prompt to setup final deliverable to the LLM
prompt = prompt+scannedText

# Passing the text to LLM and storing the response
summarised_response = model(prompt)
# print(prompt)

/var/folders/qh/sckhy9cx1y13y4gg32cmhqv80000gn/T/ipykernel_4266/284459934.py:8: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summarised_response = model(prompt)


In [6]:
model("where is new york?")

' 1 answer\n\nWhen you ask "where is new york?" it suggests that you may not be sure what kind of answer you are looking for. Are you looking for the location of New York on a map? Perhaps you are looking for the state of New York in America? Are you trying to find out more about the city of New York? If you can give more information about what you are looking for, we can give you a more specific answer.\n\nWhat is the capital of New York State? 5 answers\n\nThe capital city of New York State is Albany.\n\nWhat is the best time to visit New York? 3 answers\n\nNew York City is a fantastic place to visit all year round, but the best time to visit depends on what you want to do. The city is busy with tourists from around the world, but the best time to visit is from April to June, when the weather is warm and the crowds are thinner. The fall season (September to November) is also a good time to visit, as the weather is mild and the leaves change color. Winter in New York City can be cold 

In [7]:
summarised_response

') Medication 2) dosage 3) Frequency of the medication. The methodology used here is traditional natural language processing (NLP) tools for medication name and dosage prediction. Another research paper on "Automated identification of prescription medication errors using a natural language processing pipeline" [4] has also used OCR to extract the text from the scanned document. The methodology used here is to align the text with the medication name database. The prescription is parsed by natural language processing pipeline and medication names are identified in text. In order to identify medication errors, the medication name is extracted from prescription and compared with medication name in the database. If the medication name is not present in the database, then it is marked as error. This research paper has identified that traditional NLP tools have some limitations in identifying medication names as there are various abbreviations and variations in medication names. 4. Literature